In [1]:
%pwd

'D:\\Work\\notebook\\metricstudy\\notebook'

In [20]:
BTC_ETH = "BTC_ETH"

poloniex_url_template = "https://poloniex.com/public?command=returnTradeHistory&currencyPair=%s&start=%s&end=%s"


In [21]:
#currency_pair: currency pair expression which is supported in poloniex, to: unixtime value
def get_url(currency_pair, end_unix):
    sixmonths = 60*60*24*30*6
    start_unix = end_unix - sixmonths
    url = poloniex_url_template%(currency_pair, start_unix, end_unix)
    return url

In [22]:
import time
import datetime

def now():
    #now = int(time.gmtime())
    now = datetime.datetime.utcnow()
    now = int(time.mktime(now.timetuple()))
    print(type(now))
    return now
#1493047494
now()

<class 'int'>


1493112759

In [23]:
url = get_url(BTC_ETH, now())
print (url)

<class 'int'>
https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_ETH&start=1477560759&end=1493112759


In [24]:
import requests
r = requests.get(url)
js = r.json()
print(js[:2])
print(js[-2:])

[{'globalTradeID': 111770492, 'amount': '1.66030200', 'rate': '0.03973989', 'date': '2017-04-25 09:32:39', 'tradeID': 23085022, 'type': 'sell', 'total': '0.06598021'}, {'globalTradeID': 111770491, 'amount': '0.06898886', 'rate': '0.03973989', 'date': '2017-04-25 09:32:39', 'tradeID': 23085021, 'type': 'sell', 'total': '0.00274160'}]
[{'globalTradeID': 111094460, 'amount': '1.01096137', 'rate': '0.03919504', 'date': '2017-04-23 23:59:38', 'tradeID': 23035024, 'type': 'sell', 'total': '0.03962467'}, {'globalTradeID': 111094418, 'amount': '0.00073198', 'rate': '0.03919504', 'date': '2017-04-23 23:59:25', 'tradeID': 23035023, 'type': 'sell', 'total': '0.00002868'}]


In [25]:
lastdate = js[-1]['date']
print(lastdate)


2017-04-23 23:59:25


In [26]:
import datetime

def get_gmttime(lastdate):
    #print(datetime.datetime.strptime(lastdate, "%Y-%m-%d %H:%M:%S"))
    endtime = int(datetime.datetime.strptime(lastdate, "%Y-%m-%d %H:%M:%S").timestamp())+(9*60*60)
    #print(endtime)
    return endtime
#1493015351
#1492857459

In [27]:
endtime = get_gmttime(lastdate)

url2 = get_url(BTC_ETH, endtime)
print (url2)

https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_ETH&start=1477439965&end=1492991965


In [28]:
import requests
r = requests.get(url2)
js2 = r.json()
print(js2[:2])
print(js2[-2:])

[{'globalTradeID': 111094418, 'amount': '0.00073198', 'rate': '0.03919504', 'date': '2017-04-23 23:59:25', 'tradeID': 23035023, 'type': 'sell', 'total': '0.00002868'}, {'globalTradeID': 111094413, 'amount': '0.00186834', 'rate': '0.03919504', 'date': '2017-04-23 23:59:23', 'tradeID': 23035022, 'type': 'sell', 'total': '0.00007322'}]
[{'globalTradeID': 110369272, 'amount': '51.46261882', 'rate': '0.03980019', 'date': '2017-04-22 13:52:54', 'tradeID': 22985025, 'type': 'sell', 'total': '2.04822200'}, {'globalTradeID': 110369250, 'amount': '2.00000000', 'rate': '0.03984998', 'date': '2017-04-22 13:52:53', 'tradeID': 22985024, 'type': 'buy', 'total': '0.07969996'}]


In [29]:
def get_batch_list(total_size, batch_size=100):
    total = [a for a in range(0, total_size)]
    print (total)
    
    res = list()
    while len(total) > 0:
        bs = min(batch_size, len(total))
        print (bs)
        a = total[:bs]
        res.append(a)
        if bs < len(total):
            b = total[bs:]
            total = b
        else:
            total = []
            break
    return res
    

In [30]:
import MySQLdb

In [31]:
len(js)

50000

In [32]:
db = MySQLdb.connect(db='poloniex', user='polo', passwd='polo')

In [33]:
curr = db.cursor()

In [36]:
#make insert statement

batch_size = 10

batch = get_batch_list(31, 5)

for bs in batch:
    query = "insert into btc_eth values "

    first = True
    for b in bs:
        item = js[b]
        #print (item)

        value = "(%d,%d,%s,%s,%s,'%s','%s')"%(item['globalTradeID'], item['tradeID'], item['total'], item['rate'], item['amount'], item['type'], item['date'])

        if first:
            first = False
        else:
            query += ','

        query += value

    print (query)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
5
5
5
5
5
5
1
insert into btc_eth values (111770492,23085022,0.06598021,0.03973989,1.66030200,'sell','2017-04-25 09:32:39'),(111770491,23085021,0.00274160,0.03973989,0.06898886,'sell','2017-04-25 09:32:39'),(111770483,23085020,0.00074999,0.03973989,0.01887272,'sell','2017-04-25 09:32:38'),(111770480,23085019,0.00200275,0.03976803,0.05036082,'buy','2017-04-25 09:32:38'),(111770479,23085018,0.00083882,0.03973989,0.02110785,'sell','2017-04-25 09:32:38')
insert into btc_eth values (111770474,23085017,0.00002130,0.03976803,0.00053571,'buy','2017-04-25 09:32:37'),(111770456,23085016,0.01109272,0.03973989,0.27913319,'sell','2017-04-25 09:32:34'),(111770455,23085015,0.06838721,0.03973998,1.72086681,'sell','2017-04-25 09:32:34'),(111770454,23085014,0.00024930,0.03973998,0.00627347,'sell','2017-04-25 09:32:33'),(111770451,23085013,0.13003716,0.03976698,3.26997837,'sell','2017-04-25 

In [26]:
for item in js:
    print (','.join([str(s) for s in item.values()]))
    limit -= 1
    if limit == 0:
        break

100.76937592,buy,23046136,111224587,0.03980534,2017-04-24 07:31:45,4.01115927
0.26627370,sell,23046135,111224585,0.03980532,2017-04-24 07:31:45,0.01059910
8.13710240,buy,23046134,111224564,0.03980536,2017-04-24 07:31:41,0.32390029
0.04289760,buy,23046133,111224563,0.03980533,2017-04-24 07:31:41,0.00170755
0.25122257,sell,23046132,111224479,0.03980534,2017-04-24 07:31:17,0.00999999
0.08995801,buy,23046131,111224476,0.03980536,2017-04-24 07:31:14,0.00358081
2.91004199,buy,23046130,111224475,0.03980533,2017-04-24 07:31:14,0.11583518
0.29000000,sell,23046129,111224471,0.03980532,2017-04-24 07:31:13,0.01154354
0.28640127,buy,23046128,111224382,0.03980534,2017-04-24 07:30:57,0.01140029
0.15815875,sell,23046127,111224379,0.03980532,2017-04-24 07:30:57,0.00629555


In [16]:
get_batch_list(18, 5)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
5
5
5
3


[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17]]

In [9]:
print(b)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94]


In [ ]:
try:
    curr.execute(query)
except (MySQLdb.Error, MySQLdb.Warning) as e:
    print ("MySQL Error [%d]: %s" % (e.args[0], e.args[1]))
except IndexError:
    print ("MySQL Error: %s" % str(e))